#### SQL запросы для создания и редактирования таблиц в БД

In [ ]:
CREATE TABLE pair_history (
    token_1 TEXT NOT NULL,
	token_2 TEXT NOT NULL,
	open_time TIMESTAMPTZ,
    close_time TIMESTAMPTZ DEFAULT now(),
	side_1 TEXT NOT NULL,
	side_2 TEXT NOT NULL,
	qty_1 DOUBLE PRECISION NOT NULL,
	qty_2 DOUBLE PRECISION NOT NULL,
	open_price_1 DOUBLE PRECISION NOT NULL,
    open_price_2 DOUBLE PRECISION NOT NULL,
    close_price_1 DOUBLE PRECISION NOT NULL,
	close_price_2 DOUBLE PRECISION NOT NULL,
	fee_1 DOUBLE PRECISION NOT NULL,
	fee_2 DOUBLE PRECISION NOT NULL,
	leverage DOUBLE PRECISION NOT NULL,
    pnl_1 DOUBLE PRECISION NOT NULL,
	pnl_2 DOUBLE PRECISION NOT NULL,
    profit DOUBLE PRECISION NOT NULL
);

CREATE TABLE current_ob (
    token TEXT NOT NULL,
	ts BIGINT NOT NULL,
	update_time TIMESTAMPTZ NOT NULL,
	bid_price_0 DOUBLE PRECISION NOT NULL,
    bid_volume_0 DOUBLE PRECISION NOT NULL,
    bid_price_1 DOUBLE PRECISION NOT NULL,
    bid_volume_1 DOUBLE PRECISION NOT NULL,
	bid_price_2 DOUBLE PRECISION NOT NULL,
    bid_volume_2 DOUBLE PRECISION NOT NULL,
	bid_price_3 DOUBLE PRECISION NOT NULL,
    bid_volume_3 DOUBLE PRECISION NOT NULL,
	bid_price_4 DOUBLE PRECISION NOT NULL,
    bid_volume_4 DOUBLE PRECISION NOT NULL,
	ask_price_0 DOUBLE PRECISION NOT NULL,
    ask_volume_0 DOUBLE PRECISION NOT NULL,
	ask_price_1 DOUBLE PRECISION NOT NULL,
    ask_volume_1 DOUBLE PRECISION NOT NULL,
	ask_price_2 DOUBLE PRECISION NOT NULL,
    ask_volume_2 DOUBLE PRECISION NOT NULL,
	ask_price_3 DOUBLE PRECISION NOT NULL,
    ask_volume_3 DOUBLE PRECISION NOT NULL,
	ask_price_4 DOUBLE PRECISION NOT NULL,
    ask_volume_4 DOUBLE PRECISION NOT NULL,
	PRIMARY KEY (token, update_time)
);

CREATE TABLE IF NOT EXISTS tick_ob_test (
    token        TEXT       NOT NULL,
    time         TIMESTAMPTZ NOT NULL,
    bid_price    DOUBLE PRECISION NOT NULL,
    ask_price    DOUBLE PRECISION NOT NULL,
    bid_size     DOUBLE PRECISION NOT NULL,
    ask_size     DOUBLE PRECISION NOT NULL
);

#### PostgreSQL

In [1]:
from datetime import datetime, timedelta
from bot.core.db.postgres_manager import DBManager
from zoneinfo import ZoneInfo
from datetime import timedelta, timezone
from time import sleep
import pandas as pd
import polars as pl
from decimal import Decimal
from bot.config.credentials import host, user, password, db_name

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

# pl.Config.set_tbl_rows(50)
# pl.Config.set_trim_decimal_zeros(True)
# pl.Config.set_float_precision(5)

In [2]:
# db_manager.clear_table('market_data_5s')

In [3]:
db_manager.get_table('system_state', df_type='polars')#.filter(pl.col('module_name') == 'ws')

module_name,update_time,update_ts
str,"datetime[μs, Europe/Moscow]",i64
"""market_analyzer""",2025-11-12 07:25:24 MSK,1762921524
"""ws""",2025-11-12 07:25:20 MSK,1762921520
"""trades_executor""",2025-11-12 07:25:24 MSK,1762921524


In [49]:
all_ = db_manager.get_orderbooks()
all_tokens = all_['token'].unique().to_list()

In [51]:
tokens = []

for token in sorted(all_tokens):
    first_date = db_manager.get_oldest_date_in_orderbook(token)
    last_date = db_manager.get_orderbooks(token, interval='5m', start_date=datetime(2025, 11, 5)).tail(1)['time'][0]

    # print(f'{token:>12}, {first_date}; {last_date}')
    
    if first_date < datetime(2025, 10, 25, 12, tzinfo=ZoneInfo("Europe/Moscow")):
        print(f'{token:>12}, {first_date}; {last_date}')
        tokens.append(token)
    #     tokens.append(token)
    # elif last_date > datetime(2025, 10, 8, tzinfo=ZoneInfo("Europe/Moscow")):
    #     pass
    # else:
    

  1INCH_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
    APT_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
    ARB_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
   ARKM_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
   BLUR_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
   CELO_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
    CHZ_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
    CRV_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
    DOT_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
   DYDX_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
   FLOW_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
   GALA_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
    GMT_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
    GRT_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+03:00
    IMX_USDT, 2025-10-11 12:00:00+03:00; 2025-11-12 07:35:00+0

In [53]:
len(tokens)

40

In [ ]:
add_pair_order(self, token_1, token_2, created_at, side_1, side_2, qty_1, qty_2,
                       price_1, price_2, usdt_1, usdt_2, rpnl_1, rpnl_2, upnl_1, upnl_2,
                       leverage, profit_1, profit_2, profit, status)

In [ ]:
current_pairs = db_manager.get_table('current_pairs', df_type='polars')
current_pairs

In [ ]:
db_manager.delete_pair_order('CELO_USDT', 'CRV_USDT')

In [ ]:
db_manager.clear_table('current_pairs')

In [ ]:
db_manager.get_tick_ob(token='XRP_USDT')

In [ ]:
pl.read_parquet('./data/pair_backtest/APT_KSM_1h_lr.parquet')

In [ ]:
# db_manager.clear_old_data('tick_ob', column='time', expiration_time=192, units='hours')

In [ ]:
# db_manager.get_public_trades(exchange='bybit', token='ETH_USDT', return_type='polars')

#### Orders & trading_history

In [ ]:
db_manager.clear_table('trading_history')
db_manager.clear_table('current_orders')
# redis_order_manager.clear_orders_table()

In [ ]:
db_manager.delete_order('ADA_USDT', 'okx', 'linear')

In [ ]:
db_manager.place_order(token='ADA_USDT', exchange='okx', market_type='linear', order_type='market', order_side='buy',
                  qty=160, price=0.5927, usdt_amount=94.672, realized_pnl=-0.0025, leverage=2)

In [ ]:
db_manager.close_order(token='ADA_USDT', exchange='okx', market_type='linear', qty=160.0,
                       close_price=0.5927, close_usdt_amount=94.672, close_fee=-0.0025)

In [ ]:
current_pairs = db_manager.get_table('pairs')
current_pairs

In [ ]:
current_orders = db_manager.get_table('current_orders', df_type='polars')
current_orders

In [ ]:
trading_history = db_manager.get_table('trading_history', df_type='polars')
trading_history.select('created_at', 'closed_at', 'token', 'order_side', 'open_price', 'close_price', 
                       'open_usdt_amount', 'close_usdt_amount', 'qty', 'leverage', 'realized_pnl', 'profit')

In [ ]:
trading_history['profit'].sum()

In [ ]:
calculate_profit(open_price=0.4854, close_price=0.4849, n_coins=411.5, usdt_amount=None, side='long', fee_rate=0.00055)

#### Redis

In [ ]:
%load_ext autoreload
%autoreload 2

import redis
from redis.exceptions import ConnectionError
import polars as pl
import pandas as pd
from datetime import datetime
from decimal import Decimal
import orjson
import json

redis_client = redis.Redis(db=0, decode_responses=True)

from jaref_bot.db.redis_manager import RedisManager

redis_manager = RedisManager(db_name='orderbooks')

try: 
    redis_client.ping()
    print('Сервер Redis запущен')
except ConnectionError:
    print('Сервер Redis не отвечает')

In [ ]:
redis_client.flushdb()

In [ ]:
redis_client.keys()

In [ ]:
ts = redis_client.hget("orderbook:bybit:linear:ZBCN_USDT:update_time", 'cts')
datetime.fromtimestamp(ts)

In [ ]:
redis_client.hgetall("orderbook:bybit:linear:ZBCN_USDT")

In [ ]:
dd = redis_manager.get_orderbooks(5).drop('exchange', 'market_type').rename({'symbol': 'token'})
dd

In [ ]:
db_manager.add_orderbook_bulk(dd)

In [ ]:
db_manager.get_table('current_ob')